




**Below is a problem that uses multitask evolution to solve two separate problems simultaneously. Each problem needs to find the shortest path starting from point 0, going through the remaining points and returning to point 0. The graphs of these two problems are separate.**



The algorithm starts by creating two weight matrices for two problems.

In [ ]:
import numpy as np

num_vertices = 50


weight_matrix_1 = np.random.randint(1, 10, size=(num_vertices, num_vertices))

np.fill_diagonal(weight_matrix_1, 0)

weight_matrix_2 = np.random.randint(1, 10, size=(num_vertices, num_vertices))

np.fill_diagonal(weight_matrix_2, 0)

print(weight_matrix_1)
print(weight_matrix_2)


[[0 8 8 ... 8 6 8]
 [8 0 2 ... 2 7 9]
 [2 3 0 ... 5 1 8]
 ...
 [4 1 8 ... 0 3 1]
 [7 5 3 ... 6 0 6]
 [8 6 6 ... 3 7 0]]
[[0 7 6 ... 1 9 2]
 [6 0 8 ... 5 9 1]
 [1 7 0 ... 8 4 8]
 ...
 [2 5 8 ... 0 7 4]
 [7 9 2 ... 1 0 5]
 [9 8 3 ... 2 9 0]]


In [ ]:
# decode function
def decode(p):
    return np.argsort(p)

In [ ]:
# Function that calculates the cost of the solution
def cost(population,matrix):
    result = []
    for x in population:
        total_cost = 0
        total_cost+=matrix[0][x[0]]
        for i in range(48):
          total_cost+=matrix[x[i]][x[i+1]]
        total_cost+=matrix[x[48]][0]
        result.append(total_cost)
    return np.array(result)

In [ ]:
def rank(population,matrix):
    return np.argsort(np.argsort(cost(population,matrix)))

Initialize the population,calculate skill factor.

In [ ]:
def init():
    cur_pop = np.array([np.random.permutation(49) + 1 for _ in range(50)])
    rank_1 = rank(cur_pop,weight_matrix_1)
    rank_2 = rank(cur_pop,weight_matrix_2)
    skill_factor = []
    for i in range(50):
        if rank_1[i] < rank_2[i]:
            skill_factor.append(0)
        else:
            skill_factor.append(1)
    return cur_pop, np.array(skill_factor)

Hybrid function and mutation in a population.

In [ ]:
import numpy as np

rmp = 0.3

def assortative_mating(cur_pop, skill_factor):
    offsprings = []
    offsprings_skill_factor = []
    for i in range(25):
        rnd = np.random.rand(1)[0]
        random_indices = np.random.choice(np.arange(len(cur_pop)), size=2, replace=False)
        if skill_factor[random_indices[0]] == skill_factor[random_indices[1]] or rnd < rmp:
            offspring = cur_pop[random_indices[0]][:20]
            for j in range(20,49):
              if cur_pop[random_indices[1]][j] not in offspring:
                offspring=np.append( offspring,cur_pop[random_indices[1]][j])
            for j in range(0,20):
              if cur_pop[random_indices[1]][j] not in offspring:
                offspring=np.append( offspring,cur_pop[random_indices[1]][j])
            offsprings.append(offspring)
            random_skill_factor = np.random.rand(1)[0]
            if random_skill_factor < 0.5:
                offsprings_skill_factor.append(skill_factor[random_indices[0]])
            else:
                offsprings_skill_factor.append(skill_factor[random_indices[1]])
        else:
            random_numbers = np.random.choice(np.arange(0, 49), size=2, replace=False)
            offspring0 = cur_pop[random_indices[0]].copy()
            offspring1 = cur_pop[random_indices[1]].copy()
            k0=offspring0[random_numbers[0]]
            offspring0[random_numbers[0]] = offspring0[random_numbers[1]]
            offspring0[random_numbers[1]] =k0
            k1=offspring1[random_numbers[0]]
            offspring1[random_numbers[0]] = offspring1[random_numbers[1]]
            offspring1[random_numbers[1]] =k1
            offsprings.append(offspring0)
            offsprings_skill_factor.append(skill_factor[random_indices[0]])
            offsprings.append(offspring1)
            offsprings_skill_factor.append(skill_factor[random_indices[1]])
    return np.array(offsprings), np.array(offsprings_skill_factor)

Function update scalar fitness.

In [ ]:
def update_scalar_fitness(p, s):
    matrix1_pop = p[np.where(s == 0)[0]]
    rank_matrix1 = rank(matrix1_pop,weight_matrix_1)
    scalar_fitness_matrix1 = [1/(i+1) for i in rank_matrix1]

    matrix2_pop = p[np.where(s == 1)[0]]
    rank_matrix2 = rank(matrix2_pop,weight_matrix_2)
    scalar_fitness_matrix2 = [1/(i+1) for i in rank_matrix2]
    scalar_fitness = np.random.rand(p.shape[0])
    scalar_fitness[np.where(s == 0)[0]] = scalar_fitness_matrix1
    scalar_fitness[np.where(s == 1)[0]] = scalar_fitness_matrix2
    filter_indices = np.argsort(scalar_fitness)[:25]
    fittest_pop = np.delete(p, filter_indices, axis=0)
    fittest_skill_factor = np.delete(s, filter_indices)
    return fittest_pop, fittest_skill_factor


In [ ]:
def main():
    cur_pop, skill_factor = init()
    generation = 1
    for i in range(generation):
        offspring_pop, offspring_skill_factor = assortative_mating(cur_pop, skill_factor)
        intermediate_pop = np.vstack((cur_pop, offspring_pop))
        intermediate_skill_factor = np.concatenate((skill_factor, offspring_skill_factor))
        cur_pop, skill_factor = update_scalar_fitness(intermediate_pop, intermediate_skill_factor)
        matrix1_pop = cur_pop[np.where(skill_factor == 0)[0]]
        matrix2_pop = cur_pop[np.where(skill_factor == 1)[0]]
    print("Matrix1: ", cost(matrix1_pop,weight_matrix_1).min())
    print("Matrix2: ", cost(matrix2_pop,weight_matrix_2).min())

In [ ]:
main()

Matrix1:  202
Matrix2:  204


Reference document: [Abhishek Gupta, Yew-Soon Ong, and Liang Feng -
 Multifactorial Evolution: Towards Evolutionary Multitasking](https://ieeexplore.ieee.org/abstract/document/7161358)